In [1]:
%load_ext tclmagic

#  If the prev line doesnt work  install it using the next line in your unix terminal
#  /depot/Python/Python-3.8.0/bin/pip install -U tcl-magic
#  /depot/Python/Python-3.8.0/bin/pip install -U notebook
#  You can configure it to ~/.local/ 

In [2]:
%%tcl 

set SHELLTOOL_LOC "/remote/cad-rep/msip/tools/Shelltools"
lappend auto_path "$SHELLTOOL_LOC/ddr-utils-lay/dev/lib/tcl"
lappend auto_path "/depot/tcl8.6.3/lib"

set RealBin [file dirname [file normalize [info script]] ]
set RealScript [file tail [file normalize [info script]] ]
set PROGRAM_NAME $RealScript
set LOGFILE "[pwd]/log-$PROGRAM_NAME.log"

package require Messaging 1.0
namespace import ::Messaging::*
package require Misc 1.0
namespace import ::Misc::*

package require cmdline
package require try
package require yaml

'0.3.10'


# Last Updates 02/24
- The YML files has been updated to the last version format. 
```yaml
calibre_verifs: true
releaseMacro: 
    dwc_ddr5phy_utility_blocks:
    - dwc_ddr5phy_pclk_routing_ac_ew
    - dwc_ddr5phy_pclk_routing_dx_ew
    dwc_ddr5phy_utility_cells:
    - dwc_ddr5phy_decapvddq_ew
    - dwc_ddr5phy_vddqclamp_ew
    dwc_ddr5phy_repeater_cells:
    - dwc_ddr5phy_pclk_rptx1
repeater_name: dwc_ddr5phy_repeater_cells
lef_diff_rel: 1.00a_pre3
releaseDefMacro: 
    - pro_hard_macro/dwc_ddr5phymaster_ew_inst
releasePhyvMacro: 
    - dwc_ddr5phy_vaaclamp_ew
    - dwc_ddr5phy_vddqclamp_ew
    - dwc_ddr5phy_decapvaa_tile
releaseShimMacro: dwc_ddr5phycmosx2_top_ew_shim
releasePmMailDist: vthareja,hdavid,guttman,jfisher
supply_pins_override: 
    dwc_ddr5phy_por_ew: M6 M10
    dwc_ddr5phycover_dx4_top_ew: M10 M14 M15 M16 MTOP MTOP-1
    dwc_ddr5phy_zcalio_ew: M10 M12
```

- It considers all the files that use the keywords and suffixes: `legal_release` `legalRelease` `.txt` `.t`
- `macroRelease` is converted from an array, with curve braces in the name, into a dictionary or Associated array
- List of keywords for vriables that have to keep their values as strings: `date|prune|layers|utility_name|repeater_name|tag|supply_pins`


In [18]:
%%tcl



'1.4.20'

# 2pm 
The tcl2yml validation was sucessfully passed
```perl
perl 23_alphaHLDepotRelease_compare_yaml_vs_tcl_legalRelease.t 
1..3
File: /slowfs/dcopt103/alvaro/GitLab/ddr-hbm-phy-automation-team/ddr-ckt-rel/dev/main/tests/unit_tests/../data/test_legal_release.yml
ymlcount: 38
ok 1 - compare tcl vs yml
ok 2 - processLegalReleaseFile tcl test
ok 3 - processLegalReleaseFile yml test
tclcount: 38  ymlcount: 38
```

# 220pm
I've modified my PATH to use first the script versions in 
```bash 
$HOME/bin/
```


## collect the methods for p4

We will need the variable  `global client`

::P4::print

In [ ]:
%%tcl
# pe print has many options and it's better to keep it as it is
proc ::P4::print {args} {
    try {    
        lassign [run_system_cmd "p4 print $args"] rsc_out rsc_err status
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}

#-----------------------------------------------------------------


::P4::da_isa_p4_file

In [ ]:
%%tcl

proc ::P4::da_isa_p4_file {P4file} {
    try {
        lassign [run_system_cmd "p4 files -e $P4file"] ifexist ifnotexist status
        if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return 0
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
            return 1
        } else {
            return 2
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}


::P4::da_is_checked_out

In [ ]:
%%tcl
# P4 opened
proc ::P4::da_is_checked_out {P4file {$client ""}} {
    if {$client==""} {global client}
    if {[info exists client(NAME)]} {
        set opt(client) "-c $client(NAME)"
    } else {
        set opt(client) ""
    }
    try {
        lassign [run_system_cmd "p4 files $opt(client) opened $P4file"] ifexist ifnotexist status
        if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return $ifnotexist
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
            return $ifexist
        } else {
            return 2
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}

::P4::da_p4_add_edit

In [ ]:
%%tcl

proc ::P4::da_p4_add_edit {clientFile {client ""}} {
    if {$client==""} {global client}
    if {[info exists client]} {
        set opt(client) "-c $client(NAME)"
    } else {
        set opt(client) ""
    }
    try {
        if {[file exists $clientFile]} {
            lassign [run_system_cmd "p4 $opt(client) edit $clientFile"] rsc_out rsc_err status
            dprint HIGH $rsc_out
            dprint HIGH $rsc_err
            return $rsc_out
        } else {
            lassign [run_system_cmd "p4 $opt(client) add -t text $clientFile"] rsc_out rsc_err status
            dprint HIGH $rsc_out
            dprint HIGH $rsc_err
            return $rsc_out
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
    return 0
}

::P4::da_p4_list

In [ ]:
%%tcl
proc ::P4::da_p4_list {P4path} {
    try {
        lassign [run_system_cmd "p4 files -e $P4path"] ifexist ifnotexist status
        if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return $ifnotexist
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
            return [split $ifexist "\n" ]
        } else {
            return 2
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}

::P4::da_p4_dirs

In [ ]:
%%tcl
proc ::P4::da_p4_dirs {P4path} {
    try {
        lassign [run_system_cmd "p4 dirs $P4path"] ifexist ifnotexist status
        if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return $ifnotexist
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
            return [split $ifexist "\n" ]
        } else {
            return 2
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}

 
 # TODO

 - create_p4_client
 - delete_p4_client
 - ~~p4 sync /root~~
 - ~~submit~~
 - ~~fstat~~
 - annotate
 - revert
 - where
 - have
 - filelog
 

::P4::da_create_p4_client

In [ ]:
%%tcl
return PENDING
proc ::P4::da_create_p4_client {} {
    try {
        lassign [run_system_cmd "p4 dirs $P4path"] ifexist ifnotexist status
        if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return $ifnotexist
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
            return [split $ifexist "\n" ]
        } else {
            return 2
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}


::P4::da_delete_p4_client

In [ ]:

%%tcl
return PENDING
proc ::P4::da_delete_p4_client {} {
    try {
        lassign [run_system_cmd "p4 dirs $P4path"] ifexist ifnotexist status
        if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return $ifnotexist
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
            return [split $ifexist "\n" ]
        } else {
            return 2
        }
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}

::P4::da_p4_sync_root

In [ ]:
%%tcl

proc ::P4::da_p4_sync_root {{client ""}} {
    if {$client==""} {global client}
    if {[info exists client]} {
        set opt(client) "-c $client(NAME)"
        set opt(root)   "-f $client(ROOT)"
    } else {
        set opt(client) ""
        set opt(root)   ""
    }
    try {
            lassign [run_system_cmd "p4 $opt(client) sync $opt(root) "] rsc_out rsc_err status
            dprint HIGH $rsc_out
            dprint HIGH $rsc_err
            return $rsc_out
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
    return 0
}


In [ ]:
%%tcl

proc ::P4::da_p4_submit {description P4file {client ""}} {
    if {$client==""} {global client}
    if {[info exists client]} {
        set opt(client) "-c $client(NAME)"
    } else {
        set opt(client) ""
    }
    try {
            lassign [run_system_cmd "p4 $opt(client) submit -d \"$description\" $P4file "] rsc_out rsc_err status
            dprint HIGH $rsc_out
            dprint HIGH $rsc_err
            return $rsc_out
    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
    return 0    
}




In [ ]:
%%tcl

proc ::P4::da_fstat {P4file tags {spec_tag ""}} {
	# creating variations of tags
	set tags_by_comma [regsub -all {\,  |\, |   |  | } $tags ", " ]
	set tags_list [split $tags_by_comma ","]

    try {
		# check file exists
        lassign [run_system_cmd "p4 fstat $P4file"] noTag ifnotexist status
		if {$noTag} {
			return [eprint "Tag not found: $P4file. Abort."]
		}
		# look for those tags
        lassign [run_system_cmd "p4 fstat -T $tags_by_comma $P4file"] ifexist ifnotexist status
		#check if this work and return if not
		if {$ifexist == ""} {
            dprint LOW $ifnotexist
            return $ifnotexist
        } elseif {$ifnotexist == ""} {
            dprint LOW $ifexist
			set P4filelog $ifexist
        } else {
            return 2
        }
		if {$spec_tag != ""} {
			if {[regexp -nocase {$spec_tag (\S+)} $P4filelog -> tagvalue]} {
				return $tagvalue
			}
		}
		return [veprint LOW "No value of $spec_tag in $P4filelog"]

    } on error {result options} {
        get_call_stack
        eprint $result
        eprint [dict get $options -errorstack]
    }
}